In [2]:
from data_utils.MAML_ModelNetDataLoader40 import MAMLModelNetDataset
import os
data_root = '/home/g111056119/Documents/7111056119/Pointnet_Pointnet2_pytorch/data/modelnet40_normal_resampled'

class foo:
    num_point = 1024
    use_uniform_sample = False
    use_normals = False
    num_category = 40
    pass

def split_four_categoris_normal(data_root:str, args):
    """
    將 ModelNet40 分成 4 組，每組 10 種類別
    n_way=10
    k_shot=?
    """
    file_paths = os.path.join(data_root, 'modelnet40_shape_names.txt')
    datasets = []
    with open(file_paths, 'r') as f:
        cat = [line.rstrip() for line in f]
        cat.sort()
        ii = [x * 10 for x in range(4)]
        cat = [cat[x: x + 10] for x in ii]
        datasets = [MAMLModelNetDataset(data_root, x, args) for x in cat]
    return datasets

mn = split_four_categoris_normal(data_root, foo())

In [3]:
len(mn[3])

3325

In [4]:
len(mn[3]) / 2

1662.5

In [5]:
mn[3][0]

(array([[ 0.5279 , -0.061  ,  0.5927 ],
        [-0.07676,  0.1259 , -0.8397 ],
        [-0.5386 ,  0.07028,  0.2294 ],
        ...,
        [ 0.1079 ,  0.2756 ,  0.743  ],
        [ 0.1263 ,  0.09895, -0.1691 ],
        [ 0.3001 ,  0.08873,  0.4367 ]]),
 0)

In [12]:
# Extract features and labels from the dataset
features = [item[0] for item in mn[3]]
labels = [item[1] for item in mn[3]]

features,label = [item[0],item[1] for item in mn[3]]


array([[ 0.5279 , -0.061  ,  0.5927 ],
       [-0.07676,  0.1259 , -0.8397 ],
       [-0.5386 ,  0.07028,  0.2294 ],
       ...,
       [ 0.1079 ,  0.2756 ,  0.743  ],
       [ 0.1263 ,  0.09895, -0.1691 ],
       [ 0.3001 ,  0.08873,  0.4367 ]])

In [13]:
features, labels = zip(*[(item[0], item[1]) for item in mn[3]])

In [16]:
from sklearn.model_selection import train_test_split
features, labels = zip(*[(item[0], item[1]) for item in mn[3]])
query_set_train_x, query_set_train_y, query_set_test_x, query_set_test_y= train_test_split(features , labels, test_size=0.5, random_state=42)


In [17]:
len(query_set_train_x)

1662

In [18]:
len(query_set_test_x)

1662

In [10]:
# from sklearn.model_selection import train_test_split
# train_support_set, train_query_set, test_support_set, test_query_set= train_test_split(mn[3][0],mn[3][1],test_size=0.5,random_state=42)
spilt_ratio=0.5
test_support_set=mn[3][-1]
test_support_set

(array([[-0.2078, -0.1768,  0.6402],
        [ 0.1141,  0.728 , -0.6325],
        [-0.2036, -0.4413, -0.8096],
        ...,
        [-0.1578, -0.1079,  0.2134],
        [-0.1578, -0.137 , -0.7469],
        [-0.1221,  0.5634,  0.2507]]),
 9)

In [15]:
dataset_finetune=mn[3]
# Define the sizes for the training and testing sets
query_set_train_size = int(0.5* len(dataset_finetune))  # 50% for training
query_set_test_size = len(dataset_finetune) - query_set_train_size   # 50% for testing
query_set_train_size

1662

In [12]:
from torch.utils.data import DataLoader,random_split

# Define the sizes for the training and testing sets
support_set_size = int(0.5* len(dataset_finetune))  # 50% for training
query_set_size = len(dataset_finetune) - support_set_size   # 50% for testing

# test database 再分 train(support set) >> fine tune & test(query set) >> cal accuracy
support_dataset, query_dataset = random_split(dataset_finetune, [support_set_size, query_set_size])


In [14]:
support_dataset[-1]

(array([[ 0.07073 ,  0.3976  ,  0.3626  ],
        [-0.009874, -0.9291  , -0.1479  ],
        [-0.08086 ,  0.928   , -0.3612  ],
        ...,
        [ 0.3398  , -0.4422  ,  0.09375 ],
        [-0.232   ,  0.3262  ,  0.2842  ],
        [ 0.2172  ,  0.8772  ,  0.196   ]]),
 7)

In [16]:
from torch.utils.data import DataLoader,random_split

# Define the sizes for the training and testing sets
train_ratio = 0.5
support_set_size = int(train_ratio* len(dataset_finetune))  # 50% for training
# query_set_size = len(dataset_finetune) - support_set_size   # 50% for testing
print(support_set_size)
# test database 再分 train(support set) >> fine tune & test(query set) >> cal accuracy
support_dataset, query_dataset = dataset_finetune[:support_set_size], dataset_finetune[support_set_size:]

1662


ValueError: could not convert string '/home/g111056119/Documents/7111056119/Pointnet_Pointnet2_pytorch/data/modelnet40_normal_resampled/s to float64 at row 0, column 1.

In [ ]:
from torch.utils.data import DataLoader

data_root = '/home/g111056119/Documents/7111056119/Pointnet_Pointnet2_pytorch/data/modelnet40_normal_resampled'
from data_utils.MAML_ModelNetDataLoader40 import split_four_categoris
datasets = split_four_categoris(data_root, foo())
dataset_finetune = datasets[3]
from sklearn.model_selection import train_test_split
features, labels = zip(*[(item[0], item[1]) for item in dataset_finetune])
query_set_train_x, query_set_train_y, query_set_test_x, query_set_test_y= train_test_split(features , labels, test_size=0.5, random_state=42)


# datasets = datasets[0:3]
task_data_loader = [DataLoader(x, batch_size=4, shuffle=True, num_workers=10, drop_last=True) for x in dataset_finetune]

from models.pointnet2_cls_msg import get_model as pointnet2_model
from models.pointnet2_cls_msg import get_loss as pointnet2_loss_fn
model = pointnet2_model(10, False).to("cuda")
criterion = pointnet2_loss_fn()
meta_lr = 1e-3
meta_optimizer = torch.optim.Adam(model.parameters(), lr=meta_lr)

start_epoch = 0
loop_count = 1000 // 10
for epoch in range(start_epoch, 100):
    task_iter = [iter(x) for x in task_data_loader]
    task_lr = 1e-1
    for lc in range(loop_count):
        task_model_dict = []
        task_model_loss = []
        task_points = []
        task_labels = []
        for (i, ti) in enumerate(task_iter):
            try:
                points, labels = next(ti)
            except StopIteration:
                task_iter[i] = iter(task_data_loader[i])
                points, labels = next(task_iter[i])
            points = maml_points_do_staff(points)
            labels = labels.cuda()
            task_points.append(points)
            task_labels.append(labels)